---
syncID: 
title: "Querying Location Data with NEON API and Python"
description: "Querying the 'locations/' NEON API endpoint with Python and navigating the response"
dateCreated: 2020-04-24
authors: Maxwell J. Burner
contributors: 
estimatedTime:
packagesLibraries: requests, json
topics: api
languagesTool: python
dataProduct: DP1.10003.001
code1: 
tutorialSeries: python-neon-api-series
urlTitle: neon_api_04_locations
---

In this tutorial we will learn about querying the 'locations/' endpoint of the NEON API using Python.

<div id="ds-objectives" markdown="1">

### Objectives
After completing this tutorial, you will be able to:

* Query the locations endpoint of the NEON API for data on specific NEON locations
* Parse and navigate responses from the locations endpoint of the NEON API
* Get spatial and geolocation data about NEON sites and plots
* Navigate the parent-child relationships between NEON locations


### Install Python Packages

* **requests**
* **json**
* **pandas**



</div>

In this tutorial we will learn to use the *locations/* endpoint of the NEON API to get geoLocation information about NEON sites. While the previous tutorials all covered getting information from the *sites/*, *products/*, and *data/* endpoints, we will now begin exploring other endpoints of the NEON API.

Looking through some of the data we obtained in previous tutorials, you might have noticed that references to location tend to be vague; location names and labels are used without, but lack geospatial information such as the geographic coordinates or size of the location. For more detail, we can use the *locations/* endpoint of the NEON API to get spatial information on specific locations used in NEON data collection.

## Get a Named Location

The *locations/* endpoint is usually used to provide context to locations referenced in NEON data products. Let's start by looking at the bird count data for Lower Teakettle (TEAK) again.

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
#Define API call componenets
SERVER = 'http://data.neonscience.org/api/v0/'
SITECODE = 'TEAK'
PRODUCTCODE = 'DP1.10003.001'

Again, let's get the basic bird count data for June 2018. We read the data file to a dataframe, then check the columns' names and data types.

In [ ]:
#Request data file list for 2018-04
data_request = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2018-06')
data_json = data_request.json()

In [ ]:
for file in data_json['data']['files']:
    if('count' in file['name']):
        if('basic' in file['name']):
            bird_url = file['url']

df_bird = pd.read_csv(bird_url)
print(df_bird.dtypes)


Notice the 'namedLocation' attribute/column in our dataframe. Let's look at some of the values.

In [ ]:
df_bird['namedLocation'][0:5]

## Querying Locations

We can query the location values found in this column, for example 'SJER_013.birdGrid.brd', using the *locations/* endpoint in the NEON API. This will allow us to obtain detailed geospatial data on that particular location. 

First we'll set up the request and read in the JSON data.

In [ ]:
loc_request = requests.get(SERVER+'locations/'+'TEAK_010.birdGrid.brd')
loc_json = loc_request.json()

Similar to other JSON API responses, this contains a single 'data' element which is a dictionary. Let's also look at the keys of 'data'.

In [ ]:
for key in loc_json['data'].keys():
    print(key)

First off, the data includes some basic titles and categories for the location.

In [ ]:
print('Description: ',loc_json['data']['locationDescription'])
print('Name: ',loc_json['data']['locationName'])
print('Type: ',loc_json['data']['locationType'])
print('Domain: ',loc_json['data']['domainCode'])

The actual spatial location data includes latitude, longitude, elevation, and Universal Transverse Mercator (UTM) Coordinates.

In [ ]:
print('Latitude: ',loc_json['data']['locationDecimalLatitude'])
print('Longitude: ',loc_json['data']['locationDecimalLongitude'])
print('Elevation: ',loc_json['data']['locationElevation'])
print('UTM Easting: ',loc_json['data']['locationUtmEasting'])
print('UTM Northing: ',loc_json['data']['locationUtmNorthing'])
print('Hemisphere: ', loc_json['data']['locationUtmHemisphere'])
print('UTM Zone: ', loc_json['data']['locationUtmZone'])

Let's take a closer look at the 'locationProperties' element. This is a list of dictionaries, where each dict has one 'locationPropertyName' element and a corresponding 'locationPropertyValue' element. These together provide a more detailed description of the properties of the location.

In [ ]:
#Print location property names and values
for locationProperty in loc_json['data']['locationProperties']:
    print(locationProperty['locationPropertyName'][9:], #trim 'Value for ' off beginning of each locationPropertyName
          ': ',locationProperty['locationPropertyValue'])

Finally there's the name and URLs for the locations' parent location and children locations, if any. The parent location, in this case 'TEAK', is a location of which the current location is a part.

In [ ]:
#Print name and API url of parent location
print(loc_json['data']['locationParent'], loc_json['data']['locationParentUrl'])

The children locations are smaller areas within the current location; in our example these are points within the bird grid plot. Requesting data on the children locations through the *locations/* endpoint allows us to get spatial data on a finer resolution.

In [ ]:
#Print names and API urls of child locations
for child in zip(loc_json['data']['locationChildren'], loc_json['data']['locationChildrenUrls']):
    print(child[0], child[1])